该文件是一个赋值文件，用于计算一些值

# 计算阈值

计算结果1的阈值

In [11]:
#计算结果一的阈值
from MyModel import Classification
from HaoChiUtils import DataAnalyzer as DA
import os
def get_result1_threshold(file_path):
    label_list=['快乐','恐惧','愤怒','惊讶','喜爱','厌恶','悲伤']
    
    weight_list=[-1,0.5,1,0,-1.5,0.5,1.3]
    # 初始化模型
    myClassification=Classification("bert_model")
    # 读取文本
    fir_list=os.listdir(file_path)

    total_result1=0
    for i in fir_list:
        data = DA.get_dataList(file_path+'/'+i,min_len=6)  # 获取文本数据
        # 预测
        pre=myClassification.get_predict_result(data)  # 使用模型进行预测
        res_dict=DA.calculate_label_proportions(pre,label_list=label_list)  # 计算预测结果中各标签的比例
        result1=0
        # print("res_dict：",res_dict)
        for i in range(len(label_list)):
            result1=result1+res_dict[label_list[i]]*weight_list[i]*100  # 根据权重计算结果
        total_result1+=result1
    return total_result1/len(fir_list)*0.7  # 返回平均结果
filepath="用于计算结果1阈值和熵率阈值的用户文本"
res=get_result1_threshold(filepath)
print("结果1的阈值为:",res)

结果1的阈值为: 44.82722222222222


计算熵率的阈值

In [12]:
import math
from MyModel import Classification
from HaoChiUtils import DataAnalyzer as DA
import os
#计算判断熵率高低的阈值
def get_Entropy_threshold(file_path="用于计算结果1阈值和熵率阈值的用户文本"):
    label_list=['快乐','恐惧','愤怒','惊讶','喜爱','厌恶','悲伤']
    label_dict = {label: 0 for label in label_list}
    # 初始化模型
    myClassification=Classification("bert_model")
    # 读取文本
    fir_list=os.listdir(file_path)
    # total_result1=0
    for i in fir_list:
        data = DA.get_dataList(file_path+'/'+i,min_len=6)  # 获取文本数据
        # 预测
        pre=myClassification.get_predict_result(data)  # 使用模型进行预测
        res_dict=DA.calculate_label_proportions(pre,label_list=label_list)  # 计算预测结果中各标签的比例
        # result1=0
        # print("res_dict：",res_dict)
        for i in range(len(label_list)):
            label_dict[label_list[i]]+=res_dict[label_list[i]]
        # total_result1+=result1
    for i in label_dict.keys():
        label_dict[i]=label_dict[i]/(len(fir_list))
    total_pi=0
    for i in label_list:
        if i =='惊讶':
            continue
        pi=label_dict[i]
        total_pi+=-pi*math.log2(pi)
    return total_pi/20
filepath="用于计算结果1阈值和熵率阈值的用户文本"
res=get_Entropy_threshold(filepath)
print("熵率的阈值 = ",res)

熵率的阈值 =  0.08716504273252149


计算情绪变化的阈值

In [13]:
#计算情绪变化阈值
from FunctionalInterface import TextEmotionAnalyzer as TEA 
import os
import math
#求最大最小值
def maxmin(pro_list):
    Max=0
    Min=1
    for i in pro_list:
        if i >0 and i<Min:
            Min=i
        if i>Max:
            Max=i 
    return Max,Min
#归一化,传入一个字典
def calculate_mood_change_threshold(pro_dict):
    # pro_keys=pro_dict.keys()
    pro_values=pro_dict.values()
    pro_max,pro_min=maxmin(pro_values)
    div=pro_max-pro_min
    if div ==0:
        div=1
    pro_values=[max((x-pro_min)/div,0) for x in pro_values]
    mean_u=sum(pro_values)/len(pro_values)
    S_square=0
    x_sum=0
    for x in pro_values:
        x_sum+=(x-mean_u)*(x-mean_u)
    S_square=x_sum/len(pro_values)
    S=round(math.sqrt(S_square),2)
    return S
#开始计算
tea=TEA()
res_S=[]
folder_path="用于计算情绪变化阈值的用户"
file_list=os.listdir(folder_path)
for user in file_list:
    pro_dict=tea.emotions_proportion(f"{folder_path}\\{user}",min_len=6) 
    S=calculate_mood_change_threshold(pro_dict=pro_dict)
    res_S.append(S)
mood_change_threshold=round(sum(res_S)/len(res_S),4)
print("情绪变化的阈值=",mood_change_threshold)

情绪变化的阈值= 0.35


### 计算最终评价风险等级的阈值

平均值

In [14]:
#平均值
#传入一个二维列表，是每个用户的风险等级列表
def reverse_rows(matrix):
    return [row[::-1] for row in matrix]
#计算一个用户前month个月风险平均值
def risk_mean(risk_list,month):
    if(len(risk_list)<month):
        return -1
    risk_sum=0
    Len=month 
    for i in range(Len):
        risk_sum+=risk_list[i]
    
    return round(risk_sum/Len,4)
#根据月份，计算所有用户前month个月风险平均值
def cal_mean(risks,month):
    risk_month=[]
    for risk in risks:
        risk_month.append(risk_mean(risk,month))
    cnt=0
    _sum=0
    # print(risk_month)
    for i in risk_month:
        if i != -1:
            cnt+=1
            _sum+=i
    return round(_sum/cnt,4)
#得到 近 3 6 9 个月的风险等级平均值
def get_mean_threshold(risk_list):
    #按时间逆序
    risks=reverse_rows(risk_list)
    mean_3=cal_mean(risks=risks,month=3)
    mean_6=cal_mean(risks=risks,month=6)
    mean_9=cal_mean(risks=risks,month=9)
    return mean_3,mean_6,mean_9


标准差

In [15]:
#标准差
#得到每个用户 3、6、9 月的标准差
#接着计算得到3、6、9平均标准差 三个标准差阈值
import math
#传入一个二维列表，是每个用户的风险等级列表
def reverse_rows(matrix):
    return [row[::-1] for row in matrix]
#计算一个用户前month个月风险平均值
def risk_S(risk_list,month):
    #月份不够、跳过
    if(len(risk_list)<month):
        return -1
    risk_sum=0
    Len=month 
    for i in range(Len):
        risk_sum+=risk_list[i]
    #得到平均值
    x_mean=risk_sum/Len 
    S_squre=0
    for i in range(Len):
        S_squre+=(risk_list[i]-x_mean)*(risk_list[i]-x_mean)
    return math.sqrt(S_squre/Len)
    


#根据月份，计算所有用户前month个月风险平均标准差
def cal_S(risks,month):
    risk_month=[]
    for risk in risks:
        risk_month.append(risk_S(risk,month))
    cnt=0
    _sum=0
    # print(risk_month)
    for i in risk_month:
        if i != -1:
            cnt+=1
            _sum+=i
    return round(_sum/cnt,4)
def get_S_threshold(risk_list):
    #按时间逆序
    risks=reverse_rows(risk_list)
    S_3=cal_S(risks=risks,month=3)
    S_6=cal_S(risks=risks,month=6)
    S_9=cal_S(risks=risks,month=9)
    return S_3,S_6,S_9

计算阈值

In [16]:
#根据抑郁风险用户得到阈值
from FunctionalInterface import TextEmotionAnalyzer as TEA 
tea=TEA()
import os
folder_path="用于计算风险标准差和平均值阈值的用户文本"
def get_risk_lists(folder_path):
    users=os.listdir(folder_path)
    risk_lists=[]
    for user in users:
        src_folder_path=folder_path+'\\'+user
        risk_lists.append(tea.risk_rank_list(src_folder_path=src_folder_path,min_len=6))
    return risk_lists
risk_list=get_risk_lists(folder_path)
mean_3,mean_6,mean_9=get_mean_threshold(risk_list=risk_list)
S_3,S_6,S_9=get_S_threshold(risk_list=risk_list)
print(rf"近3、6、9个月风险等级平均值阈值分别为：{mean_3}、{mean_6}、{mean_9}")
print(rf"近3、6、9个月风险等级平均标准差阈值分别为：{S_3}、{S_6}、{S_9}")

近3、6、9个月风险等级平均值阈值分别为：0.7719、1.0238、0.9012
近3、6、9个月风险等级平均标准差阈值分别为：0.81、0.8342、0.8582


In [17]:
def score_weighting(score_3,score_6,score_9):
    pass

生成词云


In [18]:
import jieba
from wordcloud import WordCloud
from HaoChiUtils import DataPreprocess as DP
from collections import Counter
import os
dp=DP(stopwords_file_path="stop_words.txt")
def get_wordcloud(folder_path):
    folders_list=os.listdir(folder_path)
    strs=" "
    for folder in folders_list:
        user_text_list=os.listdir(folder_path+'\\'+folder)
        print(folder)
        for user_text in user_text_list:

            file_path=folder_path+'\\'+folder+'\\'+user_text
            with open(file_path,'r',encoding='utf-8') as f:
                cnt=0
                for line in f:
                    cnt+=1
                    text=dp.text_clean(line,keep_segmentation=False)
                    text=text.split(' ')
                    text=[i for i in text if i!='' and len(i)>=2]
                    # print(text)
                    strs+=' '.join(text)
    return strs
folder_path="制作词云\正常用户"
seg_text=get_wordcloud(folder_path)
# seg_list=seg_text.split(' ')
# word_freq = Counter(seg_list)
# top_n=50
# top_words=dict(word_freq.most_common(top_n))
# # 创建词云对象并生成词云
# wordcloud = WordCloud(font_path='myttf.ttf', width=800, height=400,collocations=False).generate_from_frequencies(top_words)

# # 显示词云
# wordcloud.to_image()

# # 显示词云
# # wordcloud.to_image()

万迪羊肉火锅
今渡百川
仙泷小柒_
低级普男
你的镜仔气网友版
冒牌写手
我脑袋瓜嗡嗡的
扛不住墙头马上大火
是林需药
猫猫张圆圆
绝对不能被发现的微博小号
都好说但是要先给钱


In [19]:
color=["Blues","Reds","Greens","Oranges","Purples","RdYlBu","viridis",'spring','summer','autumn','winter']
seg_list=seg_text.split(' ')
word_freq = Counter(seg_list)
top_n=80
top_words=dict(word_freq.most_common(top_n))
# 创建词云对象并生成词云
# wordcloud = WordCloud(font_path='FZYTK.ttf',background_color='white', width=800, height=400,collocations=True,colormap='summer',color_func=lambda *args, **kwargs: "black").generate_from_frequencies(top_words)
for i in range(len(color)):
    wordcloud = WordCloud(font_path='myttf.ttf',background_color='black', width=1000, height=600,collocations=True,colormap=color[i]).generate_from_frequencies(top_words)

    # 显示词云
    # wordcloud.to_image()

    # 显示词云
    # wordcloud.to_image()
    wordcloud.to_file(f"词云\正常用户\\{i+1}.jpg")

In [20]:
# import matplotlib.pyplot as plt
# wd_0=WordCloud(font_path='myttf.TTF',
#                colormap='viridis',
#                 width=800,
#                 height=400,
#                collocations=False
#                ).generate(seg_text)
# plt.imshow(wd_0,interpolation='bilinear')
# plt.axis('off')
# plt.show

In [21]:
# #将一组文本整合成一个
# folder_path="用于计算阈值的用户"
# fir_dir=os.listdir(folder_path)
# for user in fir_dir:
#     with open(f"{folder_path}\\{user}.txt",'w',encoding='utf-8') as f_write:
#         user_path=folder_path+'\\'+user 
#         print(user_path)
#         months=os.listdir(user_path)
#         for month in months:
#             user_text_path=user_path+'\\'+month
#             with open(user_text_path,'r',encoding='utf-8') as f_read:
#                 for line in f_read:
#                     line=line.strip()
#                     f_write.write(line+'\n')


In [3]:
from HaoChiUtils import DataAnalyzer as DA
from FunctionalInterface import TextEmotionAnalyzer as TEA
import os
#计算得到占比
tea=TEA()
file_path="计算各项阈值的文本\用于计算结果1阈值和熵率阈值的用户文本"
users=os.listdir(file_path)
for user in users:
    user_path=file_path+'\\'+user
    data_list=DA.get_dataList(user_path,min_len=6)
    pro_dict=tea.dri.get_pro_dict(data_list)
    print(user)
    print(pro_dict)

c:\Users\12631\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:1897: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
c:\Users\12631\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:1897: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
c:\Users\12631\AppData\Local\Programs\Python\Python39\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2287: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
c:\Users\12631\AppData\Local\Programs\Python\Python39\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1859: UserWarning: Truncation was not explicitly activated but `max_length` is provided a

ammm18_20230117 20230629 .txt
{'快乐': 0.02, '恐惧': 0.0, '愤怒': 0.09, '惊讶': 0.04, '喜爱': 0.02, '厌恶': 0.06, '悲伤': 0.77}
hhfxvbhxtjcckf_20230131 20230629 .txt
{'快乐': 0.05, '恐惧': 0.02, '愤怒': 0.02, '惊讶': 0.0, '喜爱': 0.06, '厌恶': 0.01, '悲伤': 0.85}
icouldnotrunsoiflewaway_20230227 20230711 .txt
{'快乐': 0.08, '恐惧': 0.01, '愤怒': 0.07, '惊讶': 0.0, '喜爱': 0.03, '厌恶': 0.06, '悲伤': 0.75}
JHw21-_20230421 20230628 .txt
{'快乐': 0.16, '恐惧': 0.0, '愤怒': 0.0, '惊讶': 0.02, '喜爱': 0.02, '厌恶': 0.07, '悲伤': 0.72}
shsjxhdjxjd_20220726 20230731 .txt
{'快乐': 0.13, '恐惧': 0.01, '愤怒': 0.04, '惊讶': 0.0, '喜爱': 0.07, '厌恶': 0.01, '悲伤': 0.73}
S_u_bmerge_20230201 20230717 .txt
{'快乐': 0.06, '恐惧': 0.0, '愤怒': 0.12, '惊讶': 0.03, '喜爱': 0.03, '厌恶': 0.03, '悲伤': 0.72}
WHysoSeErious.txt
{'快乐': 0.17, '恐惧': 0.0, '愤怒': 0.17, '惊讶': 0.0, '喜爱': 0.25, '厌恶': 0.0, '悲伤': 0.42}
_壹然__20230407 20230721 .txt
{'快乐': 0.12, '恐惧': 0.02, '愤怒': 0.06, '惊讶': 0.0, '喜爱': 0.05, '厌恶': 0.13, '悲伤': 0.61}
东城不下雨.txt
{'快乐': 0.26, '恐惧': 0.0, '愤怒': 0.23, '惊讶': 0.0, '喜爱': 0.1, '厌恶